IMPORTING NECESSARY LIBRARIRES

In [1]:
import pandas as pd
import numpy as np
import spotipy
import warnings
import os
from dotenv import load_dotenv
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from spotipy.oauth2 import SpotifyClientCredentials
from concurrent.futures import ThreadPoolExecutor, as_completed
warnings.filterwarnings('ignore')

SPOTIFY API CREDENTIALS

In [2]:
path = r'C:\Users\msi thin\Desktop\recommender\.env'
load_dotenv(dotenv_path=path)

True

In [3]:
cid = os.getenv('SPOTIFY_CLIENT_ID')
secret = os.getenv('SPOTIFY_CLIENT_SECRET')
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

LOADING THE PREPROCESSED AND CLEANED DATASET

In [4]:
df = pd.read_csv(r'C:\Users\msi thin\Desktop\recommender\final_data.csv')
df

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,language
0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3,6
1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4,6
2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4,6
3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4,6
4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160193,Kanye West,I Love It (& Lil Pump),4S8d14HvHb70ImctNgVzQQ,72,2018,chicago rap,0.901,0.522,2,-8.304,1,0.3300,0.0114,0.000000,0.2590,0.329,104.053,127946,4,6
1160194,Rihanna,FourFiveSeconds,78TTtXnFQPzwqlbtbwqN0y,80,2015,pop,0.582,0.272,2,-5.662,1,0.0501,0.8750,0.000000,0.1300,0.354,205.846,188238,4,6
1160195,Kanye West,FML,30Zcjs7pgEkmJA1lEbGSWT,64,2016,rap,0.527,0.415,5,-7.184,1,0.1310,0.3290,0.000000,0.1950,0.120,125.357,236120,4,6
1160196,¥$,PAPERWORK,2yyO7EKRr7c3txi4xCXUFk,77,2024,chicago rap,0.741,0.488,8,-5.308,1,0.0556,0.0703,0.000009,0.0821,0.186,120.839,145785,4,6


ENCODING & SCALING

In [5]:
label_encoder = LabelEncoder()
df['genre_encoded'] = label_encoder.fit_transform(df['genre']) #encoding the genre column to get rid of the string values

features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo', 
            'time_signature', 'popularity', 'genre_encoded', 'language'] #these features were chosen by testing the impact of each one

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])
df[features] #we can see that the values are scaled & encoded

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity,genre_encoded,language
0,0.486405,0.303,0.747479,0.044181,0.696787,0.000000,0.1150,0.139,0.533639,0.6,0.68,0.000000,1.0
1,0.576032,0.454,0.743932,0.026571,0.478916,0.000014,0.0974,0.515,0.560744,0.8,0.50,0.000000,1.0
2,0.411883,0.234,0.690643,0.033265,0.339357,0.000050,0.0895,0.145,0.559344,0.8,0.57,0.000000,1.0
3,0.394763,0.251,0.750794,0.037384,0.810241,0.000000,0.0797,0.508,0.819867,0.8,0.58,0.000000,1.0
4,0.433031,0.791,0.819657,0.031102,0.072892,0.019300,0.1100,0.217,0.687475,0.8,0.54,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160193,0.907351,0.522,0.774770,0.339856,0.011446,0.000000,0.2590,0.329,0.416224,0.8,0.72,0.096491,1.0
1160194,0.586103,0.272,0.815876,0.051596,0.878514,0.000000,0.1300,0.354,0.823407,0.8,0.80,0.631579,1.0
1160195,0.530715,0.415,0.792196,0.134912,0.330321,0.000000,0.1950,0.120,0.501442,0.8,0.64,0.692982,1.0
1160196,0.746224,0.488,0.821384,0.057261,0.070582,0.000009,0.0821,0.186,0.483370,0.8,0.77,0.096491,1.0


In [6]:
genre_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
genre_mapping #to find the key for 'pop' which will be used as a default genre if no genre is specified for a song

{'acoustic': 0,
 'afrobeat': 1,
 'alt-rock': 2,
 'ambient': 3,
 'azeri classical': 4,
 'azeri pop': 5,
 'azeri traditional': 6,
 'black-metal': 7,
 'blues': 8,
 'breakbeat': 9,
 'cantopop': 10,
 'chicago rap': 11,
 'chicago-house': 12,
 'chill': 13,
 'classic azeri pop': 14,
 'classical': 15,
 'club': 16,
 'comedy': 17,
 'country': 18,
 'dance': 19,
 'dancehall': 20,
 'death-metal': 21,
 'deep-house': 22,
 'detroit-techno': 23,
 'disco': 24,
 'dmv rap': 25,
 'drum-and-bass': 26,
 'dub': 27,
 'dubstep': 28,
 'edm': 29,
 'electro': 30,
 'electronic': 31,
 'emo': 32,
 'ethiopian pop': 33,
 'folk': 34,
 'forro': 35,
 'french': 36,
 'funk': 37,
 'gangster rap': 38,
 'garage': 39,
 'german': 40,
 'german techno': 41,
 'gospel': 42,
 'goth': 43,
 'grindcore': 44,
 'groove': 45,
 'guitar': 46,
 'hard-rock': 47,
 'hardcore': 48,
 'hardstyle': 49,
 'heavy-metal': 50,
 'hip hop': 51,
 'hip-hop': 52,
 'house': 53,
 'indian': 54,
 'indie-pop': 55,
 'industrial': 56,
 'jazz': 57,
 'jazz house': 58,


CREATING THE FEATURE MATRIX

In [7]:
df['song_vector'] = df[features].values.tolist() #we store these values in the dataframe for later use
song_matrix = df[features].values #this matrix is used to calculate cosine similarity
song_norms = np.linalg.norm(song_matrix, axis=1) #this vector is used to calculate cosine similarity
song_matrix.shape

(1160198, 13)

LANGUAGE CLASSIFICATION FUNCTION

In [8]:
def classify_language(text): #this simple function detects the langauge of the song based on the characters in its title, not perfect but enough since it's not the main focus
    char_categories = {
        'ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅏㅑㅓㅕㅗㅛㅜㅠㅡㅣ': 0,
        'あいうえおかきくけこ': 1,
        'ñ': 2,
        'éèàùìò': 3,
        'əğ': 4,
        'абвгдежзийклмнопрстуфхцчшщъыьэюя': 5
    }

    text_lower = text.lower()
    for chars, category in char_categories.items():
        if any(char in text_lower for char in chars):
            return category
    return 6

FETCHING UNKNOWN SONG FEATURES

In [9]:
def fetch_track_data(track_id): #this function is used to fetch the audio features of a song which was not found in the database
    try: #fetching the audio features of the song
        results = sp.audio_features(track_id)[0]
        audio_data = [
            results['danceability'],
            results['energy'],
            results["loudness"],
            results['speechiness'],
            results['acousticness'],
            results['instrumentalness'],
            results['liveness'],
            results['valence'],
            results['tempo'],
            results['time_signature'],
            sp.track(track_id)['popularity']
        ]
    except Exception as e: #this exception may be useful if the song in playlist is not available in spotify's database, i.e. a local file
        print('EXCEPTION: Could not retrieve audio features for this song:', e) 
        return 
    
    genres = sp.artist(sp.track(track_id)['artists'][0]['id'])['genres'] #fetching the genre of the song
    if len(genres) == 0: #if no genre is found, we use the default genre 'pop'
        genre_encoded = 72     
    else:
        genre_candidates = []
        for genre in genres:
            try:
                genre_encoded = label_encoder.transform([genre])[0] #encoding the genres 
                genre_candidates.append(genre_encoded)
            except ValueError:
                pass

        if genre_candidates: #if the genre is found, we take the average of the genres
            genre_encoded = np.mean(genre_candidates)
        else:
            genre_encoded = 72 #if the genre is not found, we use the default genre 'pop'
            
    language = classify_language(sp.track(track_id)['name'])
    audio_data.append(genre_encoded)
    audio_data.append(language)
    audio_data = np.array(audio_data).reshape(1, -1)
    audio_data = scaler.transform(audio_data) #scaling the audio features 
    return audio_data[0]  

def unknown_song_matrix(candidate_unknown_ids): #this function is used to create a matrix of the songs which were not found in the database
    unknown_song_matrix = [] #this list will store the audio features of the songs which were not found in the database
    with ThreadPoolExecutor(max_workers=50) as executor: #we use ThreadPoolExecutor to fetch the audio features of the songs concurrently
        future_to_track = {executor.submit(fetch_track_data, i): i for i in candidate_unknown_ids} #we submit the fetch_track_data function to the executor
        for future in as_completed(future_to_track): 
            data = future.result()
            if data is not None:
                unknown_song_matrix.append(data)

    return np.array(unknown_song_matrix)      

PLAYLIST INPUT & DISPLAY

In [10]:
#playlist data is fetched from the spotify API using the playlist link
playlist_link = "https://open.spotify.com/playlist/1Rbw9tvNFHHUCdtPuEnGEz?si=2fe3d1d603e1464d"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
playlist_data = sp.playlist(playlist_URI)
user_id = playlist_data["owner"]["id"]
user_profile = sp.user(user_id)

user_profile_picture = user_profile["images"][0]["url"]
username = playlist_data["owner"]["display_name"]
playlist_name = playlist_data["name"]
number_of_tracks = playlist_data["tracks"]["total"]
playlist_cover_image = playlist_data["images"][0]["url"]

playlist_tracks = sp.playlist_tracks(playlist_URI)
total_tracks = playlist_tracks["total"]
tracks = playlist_tracks["items"]

#the purpose of this while loop is to fetch all the tracks in the playlist, since the API only fetches 100 tracks at a time
#it keeps fethcing until all the tracks are fetched, offset is used to keep track of the number of tracks fetched
while len(tracks) < total_tracks:
    playlist_tracks = sp.playlist_tracks(playlist_URI, offset=len(tracks))
    tracks.extend(playlist_tracks["items"])

#since the duration is returned in milliseconds, we convert it to hours and minutes
duration = sum([track["track"]["duration_ms"] for track in tracks])
hours, minutes = duration//3600000, duration % 3600000 // 60000
duration = f"{hours} hours {minutes} minutes" if hours else f"{minutes} minutes"
 

print(f"Username: {username}")
print(f"User Profile Picture: {user_profile_picture}")
print(f"Playlist Name: {playlist_name}")
print(f"Number of Tracks: {number_of_tracks}")
print(f"Playlist Cover Image: {playlist_cover_image}")
print(f"Duration: {duration}")

track_uris = [track["track"]["uri"] for track in playlist_data["tracks"]["items"]]
ids_playlist = [uri.split(":")[-1] for uri in track_uris]

#for loop to remove the tracks which are not found in the database, i.e. local files
for i in ids_playlist:
    ids_playlist.remove(i) if len(i) != 22 else None

Username: Murad
User Profile Picture: https://i.scdn.co/image/ab67757000003b82a43719576a1a56ce2e931bbb
Playlist Name: indie
Number of Tracks: 13
Playlist Cover Image: https://mosaic.scdn.co/640/ab67616d00001e0296bc7d5bd0b6488de76598abab67616d00001e02a788765bdda04106bd7b0d29ab67616d00001e02b69321baf3e7031511fa0a93ab67616d00001e02efbe96040319db682b438f11
Duration: 41 minutes


PLAYLIST RECOMMENDATION (FIRST APPROACH)

In [21]:
candidate_known_ids = [] #this will store the audio features of the songs which were found in the database
candidate_unknown_ids = [] #this list will store the track ids of the songs which were not found in local database

for id in ids_playlist:
    try:
        input_song_vector_d = df[df['track_id'] == id]['song_vector'].values[0]
        candidate_known_ids.append(input_song_vector_d)
    except IndexError:
        candidate_unknown_ids.append(id)

candidate_known_ids = np.array(candidate_known_ids)
song_vectors = np.concatenate((candidate_known_ids, unknown_song_matrix(candidate_unknown_ids)), axis=0) #concatenating the known and unknown song vectors
song_vectors = np.array([x for x in song_vectors if x is not None]) #removing the None values from the song vectors
input_song_vector = np.mean(song_vectors, axis=0) #taking the average of the audio features of the songs in the playlist

In [22]:
TOP = 20
similarities = np.dot(song_matrix, input_song_vector) / (song_norms * np.linalg.norm(input_song_vector)) #calculating the cosine similarity
sorted_indices = np.argsort(similarities)[::-1][:TOP] #sorting the indices based on the similarity values
top_songs = df.loc[sorted_indices, ['track_name', 'artist_name', 'track_id']] #fetching the top songs based on the similarity values
top_songs['similarity'] = similarities[sorted_indices] #adding the similarity values to df
top_songs = top_songs[~top_songs['track_id'].isin(ids_playlist)] #removing the songs which are already in the playlist

#the code below is used to fetch the mp3, spotify and image urls of the top songs
mp3_urls = []
spotify_urls = []
image_urls = []
ids = top_songs['track_id'].tolist()
results = sp.tracks(ids)

for track in results['tracks']:
    mp3_urls.append(track['preview_url'])
    spotify_urls.append(track['external_urls']['spotify'])
    image_urls.append(track['album']['images'][0]['url'])

top_songs['mp3_url'] = mp3_urls #not every song has a preview url provided by Spotify, so some of the values will be None
top_songs['spotify_url'] = spotify_urls
top_songs['image_url'] = image_urls

top_songs

,track_name,artist_name,track_id,similarity,mp3_url,spotify_url,image_url
342778,Tired,beabadoobee,6F5mZpEEjhsAW8UEqbIpz1,0.996762,https://p.scdn.co/mp3-preview/e6b03ecee7d3883b...,https://open.spotify.com/track/6F5mZpEEjhsAW8U...,https://i.scdn.co/image/ab67616d0000b273fe1158...
146805,Blue Dream,Jhené Aiko,4Y8MG7qcAMk7JB7Eebpcvu,0.995215,None,https://open.spotify.com/track/4Y8MG7qcAMk7JB7...,https://i.scdn.co/image/ab67616d0000b273dc94d5...
462030,Fire On Fire,Sam Smith,7t3Xdbufg7q2onVsR8RBdY,0.994426,None,https://open.spotify.com/track/7t3Xdbufg7q2onV...,https://i.scdn.co/image/ab67616d0000b2730d1f39...
569187,Seek & Destroy,SZA,6eT2V7nKXyMf47TwPbtgAD,0.994388,https://p.scdn.co/mp3-preview/8a5d798429120b44...,https://open.spotify.com/track/6eT2V7nKXyMf47T...,https://i.scdn.co/image/ab67616d0000b27370dbc9...
399142,Without You,Strawberry Guy,2hIrHUgeI2cKYwM18MkGPK,0.993307,https://p.scdn.co/mp3-preview/e3b1578cb8e6d9da...,https://open.spotify.com/track/2hIrHUgeI2cKYwM...,https://i.scdn.co/image/ab67616d0000b27303898b...
350323,"Fire On Fire - From ""Watership Down""",Sam Smith,5rFj8OzffT0iggSjcBaeND,0.993214,None,https://open.spotify.com/track/5rFj8OzffT0iggS...,https://i.scdn.co/image/ab67616d0000b2739df54b...
293480,Drew Barrymore,SZA,06u5LrUpbosQlQ1QJFhPpG,0.993165,https://p.scdn.co/mp3-preview/98893dda53ce0bba...,https://open.spotify.com/track/06u5LrUpbosQlQ1...,https://i.scdn.co/image/ab67616d0000b2734c79d5...
286612,Only in My Dreams,The Marías,190IqlryWu91WBKeDgZqZz,0.993095,https://p.scdn.co/mp3-preview/b6443de0208492c8...,https://open.spotify.com/track/190IqlryWu91WBK...,https://i.scdn.co/image/ab67616d0000b2730924b9...
515869,Here's Your Perfect,Jamie Miller,2aTKrdenCq5qBOoJPFdn4P,0.993061,https://p.scdn.co/mp3-preview/39ad57fb35d1f3e5...,https://open.spotify.com/track/2aTKrdenCq5qBOo...,https://i.scdn.co/image/ab67616d0000b2730f77b5...
288482,"Agar Tum Saath Ho (From ""Tamasha"")",Alka Yagnik,2FCXQHugkoHE1K3tiDu8pu,0.992762,https://p.scdn.co/mp3-preview/337567e59bd647a7...,https://open.spotify.com/track/2FCXQHugkoHE1K3...,https://i.scdn.co/image/ab67616d0000b2737569cb...


SECOND APPROACH

In [23]:
candidate_known_ids = [] #this will store the audio features of the songs which were found in the database
candidate_unknown_ids = [] #this list will store the track ids of the songs which were not found in local database

for id in ids_playlist:
    try:
        input_song_vector_d = df[df['track_id'] == id]['song_vector'].values[0]
        candidate_known_ids.append(input_song_vector_d)
    except IndexError:
        candidate_unknown_ids.append(id)

candidate_known_ids = np.array(candidate_known_ids)
song_vectors = np.concatenate((candidate_known_ids, unknown_song_matrix(candidate_unknown_ids)), axis=0)
song_vectors = np.array([x for x in song_vectors if x is not None])
song_vectors_transpose = song_vectors.T #transposing the song vectors (matrix)
#calculating the cosine similarity between the songs in the database and all the songs in the playlist
similarities_matrix = np.dot(song_matrix, song_vectors_transpose) / (song_norms[:, np.newaxis] * np.linalg.norm(song_vectors_transpose, axis=0)) 
similarities_matrix_transpose = similarities_matrix.T

song_matrix.shape, song_vectors_transpose.shape

((1160198, 13), (13, 13))

In [24]:
c = {}

#this for loop is used to find the top 5 songs for each song in the playlist
#it will add the top 5 songs to the dictionary c, with the key being the index of the song in the playlist and the value being the similarity value
for vector in similarities_matrix_transpose:
    indices = np.argsort(vector)[::-1][:5]
    values = vector[indices]
    for j in range(len(indices)):
        c[indices[j]] = values[j]
        
sorted_c = dict(sorted(c.items(), key=lambda item: item[1], reverse=True))
sorted_c

{606502: 1.0000000000000002,
 235638: 1.0000000000000002,
 399050: 1.0000000000000002,
 342796: 1.0,
 235694: 1.0,
 561955: 1.0,
 569170: 0.9999999999999998,
 387878: 0.9989728925332074,
 515738: 0.99847575787182,
 342768: 0.9980606720474973,
 1057254: 0.9978387895930316,
 354051: 0.9978230087276901,
 515764: 0.997563989295458,
 515599: 0.9974953676159609,
 146797: 0.9973423722793587,
 569189: 0.9973032983827784,
 147040: 0.9967614581641943,
 77593: 0.9965361909675228,
 93574: 0.9962163214775622,
 354028: 0.9960647230989828,
 350807: 0.996024018930394,
 189459: 0.995877664392914,
 462030: 0.9958084286303713,
 350288: 0.9956284550438227,
 197580: 0.995559632161716,
 350192: 0.9954407883254445,
 329763: 0.9951528964861553,
 569190: 0.9951200603469671,
 93700: 0.9951165475918374,
 197703: 0.9950876141247189,
 293686: 0.9949292070140975,
 146790: 0.9947795317978952,
 77596: 0.9940077011499413,
 569173: 0.9939338959063823,
 232893: 0.9938936277941657,
 777859: 0.9936289194562695,
 23947: 0.

In [25]:
top_songs_new = pd.DataFrame()
indices = []
similarities = []

#this loop is used to find the top 20 songs which are not in the playlist
for key, value in sorted_c.items():
    if value < 1:
        indices.append(key)
        similarities.append(value)
    if len(indices) == 20:
        break

#the code below is used to fetch the top song data using the indices 
top_songs_new = df.loc[indices, ['track_name', 'artist_name', 'track_id']] 
top_songs_new['similarity'] = similarities
top_songs_new = top_songs_new[~top_songs_new['track_id'].isin(ids_playlist)]

mp3_urls = []
spotify_urls = []
image_urls = []
ids = top_songs_new['track_id'].tolist()
results = sp.tracks(ids)

for track in results['tracks']:
    mp3_urls.append(track['preview_url'])
    spotify_urls.append(track['external_urls']['spotify'])
    image_urls.append(track['album']['images'][0]['url'])

top_songs_new['mp3_url'] = mp3_urls
top_songs_new['spotify_url'] = spotify_urls
top_songs_new['image_url'] = image_urls

top_songs_new

,track_name,artist_name,track_id,similarity,mp3_url,spotify_url,image_url
387878,Feu rouge,Pépite,314LZAYaJXobwhsUapt0nJ,0.998973,https://p.scdn.co/mp3-preview/771ece1fb9fe5ec4...,https://open.spotify.com/track/314LZAYaJXobwhs...,https://i.scdn.co/image/ab67616d0000b273a26f63...
515738,The Moment I Knew (Taylor's Version),Taylor Swift,0NRHj8hDwwmSPaA41o379r,0.998476,None,https://open.spotify.com/track/0NRHj8hDwwmSPaA...,https://i.scdn.co/image/ab67616d0000b273318443...
342768,Show Me How,Men I Trust,01TyFEZu6mHbffsVfxgrFn,0.998061,None,https://open.spotify.com/track/01TyFEZu6mHbffs...,https://i.scdn.co/image/ab67616d0000b27340faef...
1057254,Last Night on Earth,Green Day,5TpPSTItCwtZ8Sltr3vdzm,0.997839,https://p.scdn.co/mp3-preview/966749f64abdac65...,https://open.spotify.com/track/5TpPSTItCwtZ8Sl...,https://i.scdn.co/image/ab67616d0000b273c2ced3...
354051,Birds,Imagine Dragons,6Tvzf3VEi16JMhAgOwdt2y,0.997823,None,https://open.spotify.com/track/6Tvzf3VEi16JMhA...,https://i.scdn.co/image/ab67616d0000b273da6f73...
515764,What Other People Say,Sam Fischer,2h9TDNEXRhcDIV3fsoEVq9,0.997564,https://p.scdn.co/mp3-preview/12c47066042462eb...,https://open.spotify.com/track/2h9TDNEXRhcDIV3...,https://i.scdn.co/image/ab67616d0000b273027623...
515599,Reckless,Madison Beer,5ajjAnNRh8bxFvaVHzpPjh,0.997495,https://p.scdn.co/mp3-preview/393eb37e58b0760f...,https://open.spotify.com/track/5ajjAnNRh8bxFva...,https://i.scdn.co/image/ab67616d0000b273fe32f3...
146797,Recognize (feat. Drake),PARTYNEXTDOOR,1DMYEiuAgz1OKvANXiNFrN,0.997342,https://p.scdn.co/mp3-preview/6a06f19d61464c94...,https://open.spotify.com/track/1DMYEiuAgz1OKvA...,https://i.scdn.co/image/ab67616d0000b2736cfa29...
569189,Glimpse of Us,Joji,6xGruZOHLs39ZbVccQTuPZ,0.997303,https://p.scdn.co/mp3-preview/12e4a7ef3f105142...,https://open.spotify.com/track/6xGruZOHLs39ZbV...,https://i.scdn.co/image/ab67616d0000b27308596c...
147040,Ennavale Adi Ennavale,Unnikrishnan,5KfLr5UF3iQN0ifFMQH0ol,0.996761,None,https://open.spotify.com/track/5KfLr5UF3iQN0if...,https://i.scdn.co/image/ab67616d0000b2733adcf2...
